In [4]:
# import pandas as pd
# df = pd.read_csv("IT Support Ticket Data.csv")
# df['Department'].unique()

In [1]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

# --- Configuration ---
# Your model ID on Hugging Face Hub
MODEL_ID = "parth1609/gpt2-it-ticket-classifier"

# Your Hugging Face API token
# It's highly recommended to load this from an environment variable for security
# Make sure you've set HF_TOKEN="hf_YOUR_TOKEN_HERE" in your environment
# API_TOKEN = os.getenv("HF_TOKEN")
API_TOKEN = os.getenv("hf_token")

if not API_TOKEN:
    raise ValueError("Hugging Face API token not found. Please set the HF_TOKEN environment variable.")

# The Inference API URL for your model
API_URL = f"https://api-inference.huggingface.co/models/{MODEL_ID}"

# Headers for authentication
headers = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/json" # Specify content type for JSON payload
}

# --- Function to send a query to the API ---
def query(payload):

    response = requests.post(API_URL, headers=headers, json=payload)
    response.raise_for_status() # Raises an HTTPError for bad responses (4xx or 5xx)
    return response.json()


In [8]:
from transformers import pipeline
import torch

# initialize pipeline with your model
pipe = pipeline(
    "text-generation",
    model=MODEL_ID,
    tokenizer=MODEL_ID,
    device=0 if torch.cuda.is_available() else -1  # use GPU if available
)

# example ticket list
sample_tickets = [
    "My laptop is not turning on after the latest update.",
    "The VPN disconnects every time I upload files.",
    "Unable to access email - Authentication error prevents login to Outlook",
    "HP LaserJet printer offline and unresponsive to print commands",
    "Critical performance degradation on company intranet portal",
    "Print jobs stuck in queue and not processing on network printer",
    "Network printer showing offline status and connection errors",
]

for ticket in sample_tickets:
    prompt = (
        f"Classify the following IT ticket and return output in JSON format with keys: ticket', 'Department', 'Priority'.\n \"{ticket}\""
    )

    # include generation settings
    output = pipe(prompt, max_new_tokens=1024, temperature=0.7, do_sample=False)

    text = output[0]["generated_text"].strip()

    print("🔹 Ticket:")
    print(ticket)
    print("✅ Model Output:")
    print(text)
    print("-" * 60)


No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (1024). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


IndexError: index out of range in self

In [7]:
import json
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel # cite: 1
# Import your custom classes from gp2_finetune.py
from gp2_finetune import ITTicketClassifierProcessor, GPT2ITClassifier # cite: 1

# Define the Hugging Face model ID you want to use
# This is assumed to be a GPT-2 model fine-tuned for this task
MODEL_ID = os.getenv('MODEL_ID') 

# Set up device
device = torch.device("cuda") # cite: 1
print(f"Using device: {device}") # cite: 1

print(f"\n=== Performing Batch Classification with Hugging Face Model: {MODEL_ID} ===")

# 1. Initialize the Processor with the Hugging Face model ID
# This will load the tokenizer directly from the Hugging Face Hub,
# including any special tokens saved with that model.
processor = ITTicketClassifierProcessor(model_name=MODEL_ID) # cite: 1

# 2. Initialize the Classifier with the Hugging Face model ID and the processor
# This will load the GPT2LMHeadModel directly from the Hugging Face Hub.
classifier = GPT2ITClassifier(model_name=MODEL_ID, processor=processor) # cite: 1
# Move the model to the correct device
classifier.model.to(device) # cite: 1
classifier.model.eval() # Set model to evaluation mode # cite: 1

sample_queries = [ # cite: 1
    "Email not working, urgent issue", # cite: 1
    "Need new laptop for new employee", # cite: 1
    "Server maintenance required", # cite: 1
    "Password reset needed", # cite: 1
    "This is a very long and detailed issue description that requires immediate attention. The system has been experiencing intermittent connectivity problems affecting multiple users across different departments. Users report being unable to access critical business applications and experiencing slow response times. The issue started approximately 2 hours ago and is impacting productivity. Several attempts to restart affected services have been unsuccessful. Network monitoring shows increased latency and packet loss. Please investigate and resolve as this is affecting business operations. Priority level: High. Impact: Organization-wide. Previous troubleshooting steps taken: Service restarts, basic network diagnostics, and user workstation checks.", # cite: 1
    
]

results = []
print(f"Classifying {len(sample_queries)} queries...")

for i, query in enumerate(sample_queries): # cite: 1
    print(f"Processing {i+1}/{len(sample_queries)}: {query[:50]}...") # cite: 1
    # Use the classify_query method from your GPT2ITClassifier instance
    result = classifier.classify_query(query, temperature=0.3) # cite: 1
    results.append(result) # cite: 1

print("\n--- Classification Results ---") # cite: 1
for result in results: # cite: 1
    print(json.dumps(result, indent=2)) # cite: 1
    print("-" * 60) # cite: 1

Using device: cuda

=== Performing Batch Classification with Hugging Face Model: None ===


OSError: None is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`